&copy;Copyright for Shuang Wu, 2017<br>
cite from the cs231n, web:http://cs231n.github.io/<br>
Study notes for myself use.<br>

# CS231n

## Model 1: NN

## Image classification: Data driven approach, KNN, train/val/test splits<br>
### Image Classification
#### Motivation:<br>
Introduce the Image Classification problem, assigning an input image one label from a fixed set of categories.<br>

#### Example:<br>
Take a single image and assigns probabilities to 4 labels. {cat, dot, hat, mug}<br>
THe computer see the image as all the numbers from 0 (black) to 255 (white) in 3D, RGB.<br>
![csic1](imgs/csic1.jpg)<br>
This task gaves out the probability for each label and it will belong to the highest one.<br>

#### Challenges:<br>
1. viewpoint variation
2. scale variation
3. Deformation
4. Occlusion
5. Illumination conditions
6. Background clutter
7. Intra-class variation
![csic2](imgs/csic2.jpg)<br>

#### Data-driven approach:<br>
Provide the computer w/ many examples of each class and then develop learning algorithms that look at these examples and learn about the visual appearance of each class. This named as <i>data-driven approach</i>, it relies on first accumulating a training dataset of labeled images.<br>
![csic3](imgs/csic3.jpg)<br>
Just a small example, in fact, may have thousands of categories and thousands of images in each cate.

#### The Image Classification pipeline:<br>
1. Input: Set of N images, each labeled w/ one of K different classes. <strong>Training set</strong>.<br>
2. Learning: Using training set to learn every classes look like. <strong>Training a classifier, or learning a model</strong>.<br>
3. Evaluation: Predict labels for new set of images never seen before. Compare predict label with true label.<br>

### Nearest Neighbor Classifier:<br>
Rarely used in practice. Just basic idea.<br>
<strong>Example image classification dataset: CIFAR-10</strong>. 60,000 tiny images, $32*32$. each label w/ one of 10 classes. Split to 50,000 as train and 10,000 as test.<br>
![csic4](imgs/csic4.jpg)<br>
Left: Example images from the CIFAR-10 dataset. Right: first column shows a few test images and next to each we show the top 10 nearest neighbors in the training set according to pixel-wise difference.<br>
The nearest neighbor classifier will take a test image, compare it to every single one of the training images, and predict the label of the closest training image. Above picture show the example and we found the accuracy is low.<br>

<strong>Mathmatics:</strong><br>
The simplest way is to compare the images pixel by pixel and add up all the differences. Given 2 images as vectors $I_1$, $I_2$, a reasonable choice for comparing them might be <strong>L1 distance</strong>:<br>
$$d_1(I_1,I_2)=\sum_p|I^p_1-I^p_2|$$
This can be view as:<br>
![csic5](imgs/csic5.jpg)<br>

<strong>Code:</strong><br>
$Xtr$, $50,000*32*32*3$, training images. $Ytr$, $50,000$, traning labels, 0-9.

In [ ]:
Xtr, Ytr, Xte, Yte = load_CIFAR10('data/cifar10/') # a magic function we provide
# flatten out all images to be one-dimensional
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) # Xtr_rows becomes 50000 x 3072
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3) # Xte_rows becomes 10000 x 3072
nn = NearestNeighbor() # create a Nearest Neighbor classifier class
nn.train(Xtr_rows, Ytr) # train the classifier on the training images and labels
Yte_predict = nn.predict(Xte_rows) # predict labels on the test images
# and now print the classification accuracy, which is the average number
# of examples that are correctly predicted (i.e. label matches)
print 'accuracy: %f' % ( np.mean(Yte_predict == Yte) )

The accuracy measures the fraction of predictions that were correct. The code for the simple Nearest Neighbor classifier w/ L1 distance that satisfies this template:<br>

In [3]:
import numpy as np
class NearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # the nearest neighbor classifier simply remembers all the training data
        self.Xtr = X
        self.ytr = y

    def predict(self, X):
        """ X is N x D where each row is an example we wish to predict label for """
        num_test = X.shape[0]
        # lets make sure that the output type matches the input type
        Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

        # loop over all test rows
        for i in xrange(num_test):
            # find the nearest training image to the i'th test image
            # using the L1 distance (sum of absolute value differences)
            distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
            min_index = np.argmin(distances) # get the index with smallest distance
            Ypred[i] = self.ytr[min_index] # predict the label of the nearest example
        return Ypred

The accuracy for this method is 38.6%. Like guessing at random, 10%. The human can achive 94% and CNN 95%.<br>
Another way for compute distance: <strong>L2 distance</strong>, geometric interpretation of computing the euclidean distance between 2 vec.<br>
$$d_2(I_1,I_2)=\sqrt{\sum_p(I_1^p-I^p_2)^2}$$<br>
Code like below:<br>

In [ ]:
distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis = 1))

The accuracy for this is 35.4%.<br>
<strong>L1, L2</strong> are the most commonly used special cases of a p-norm, 1-norm and 2-norm.<br>

### K-Nearest Neighbor Classifier<br>
Instead of finding the single closest image in the training set, find the top $k$ closest images, and have them vote on the label of the test image. When $k=1$, same as nearest neighbor classifier. Higher values of k have a smoothing effect makes the classifier more resistant to outliers:<br>
![csic6](imgs/csic6.jpg)<br>

### Validation sets for Hyperparameter tuning<br>
The choosing of the value k, or the chooseing of different lorm are named <strong> Hyperparameters</strong>.<br>
We can try out many different values and see what works best, but need be doen carefully. And <strong> we cannot use the test set for the purpose of tweaking hyperparameters</strong>. Otherwise, maybe overfit, not the generalization of the classifier.<br>

<strong><i>Evaluate on the test set only a single time, at the very end</i></strong><br>

The idea that do not touch the test set and choose the good hyperparameter is using the validation set, a slightly smaller training set. Can be seen as fake test set. E.g, 50,000 training split to 49,000 training and 1,000 validation. Code as follow:<br>

In [ ]:
# assume we have Xtr_rows, Ytr, Xte_rows, Yte as before
# recall Xtr_rows is 50,000 x 3072 matrix
Xval_rows = Xtr_rows[:1000, :] # take first 1000 for validation
Yval = Ytr[:1000]
Xtr_rows = Xtr_rows[1000:, :] # keep last 49,000 for train
Ytr = Ytr[1000:]
# find hyperparameters that work best on the validation set
validation_accuracies = []
for k in [1, 3, 5, 10, 20, 50, 100]:
    # use a particular value of k and evaluation on validation data
    nn = NearestNeighbor()
    nn.train(Xtr_rows, Ytr)
    # here we assume a modified NearestNeighbor class that can take a k as input
    Yval_predict = nn.predict(Xval_rows, k = k)
    acc = np.mean(Yval_predict == Yval)
    print 'accuracy: %f' % (acc,)

    # keep track of what works on the validation set
    validation_accuracies.append((k, acc))

We could plot a graph that shows which values of k work best. And then evaluate on the actual test set.<br>

<strong><i>Split your training set into training set and a validation set. Use validation set to tune all hyperparameters. At the end run a single time on the test set and report performance.</i></strong><br>

<strong>Cross-validation</strong>.<br>
When training data size is small, use cross-validation. Instead of arbitrarily picking the 1st 1000 datapoints to be the validation set, iterating over different validation sets and averaging the performance across thest for a certain value of k. E.g, 5-fold CV, split the training into 5 folds, 4 of them for training, 1 for validation. Then iterate over which fold is the validation fold, evaluate the performance and average the perfomance across the different folds.<br>
![csic7](imgs/csic7.jpg)<br>
$k=7$ works best from the above picture for particular dataset.<br>

<strong>In practice</strong><br>
Prefer to avoid CV in favor of a single validation set, CV computationally expensive. Like to use 50%-90% of the training for training and rest for validation. But depends, if hyperparameters is large, may prefer bigger validation splits, if small, prefer CV. Typical use like 3, 5, 10-fold CV.<br>
![csic8](imgs/csic8.jpg)<br>

#### Pros and Cons of Nearest Neighbor classifier<br>
Pros:<br>
1. Simple to implement and understand<br>
2. no time to train<br>
3. Approximate Nearest Neighbor algorithms and libraries exist to accelerate the nearest neighbor lookup in a dataset.<br> "These algorithms allow one to trade off the correctness of the nearest neighbor retrieval with its space/time complexity during retrieval, and usually rely on a pre-processing/indexing stage that involves building a kdtree, or running the k-means algorithm."<br>
4. Good for low-dimensional data<br>

Cons:<br>
1. computational cost at test time, compara every single training<br>
2. Cost much on test instead of training<br>
3. Rarely appropriate in image classification or high-dimensional<br>
4. affect by the general color, type of background<br>
![csic9](imgs/csic9.jpg)<br>
![csic10](imgs/csic10.jpg)<br>

### Summary

1. The image classification problem.<br>
2. Nearest Neighbor classifier.<br>
3. Validation for choosing hyperparameters.<br>
4. CV if the training is small size.<br>
5. After find the best parameter, single evaluation on the test set.<br>
6. About 40% accuracy, simple implement but expensive to evaluate on test.<br>
7. Not very good on raw pixel image which much dependet on background.<br>

### Summary: Apply KNN

1. Normalize the features in data to have 0-mean and unit variance.<br>
2. If high dimension data, using dimensionality reduction technique like PCA or Random projections.<br>
3. Split data set into train/val.<br>
4. Train and evaluate the kNN on validation for many choices of k and different distance types.<br>
5. If running too long, consider Approximate NN library.<br>
6. Takes note of the hyperparameters that gave the best results.